## Criando uma tool de busca de temperatura

In [2]:
from langchain.agents import tool
from langchain.pydantic_v1 import BaseModel, Field
import requests
import datetime


class RetornaTempArgs(BaseModel):
    latitude: float = Field(
        description="Latitude da localidade para a qual deseja obter a temperatura"
    )
    longitude: float = Field(
        description="Longitude da localidade para a qual deseja obter a temperatura"
    )

@tool(args_schema=RetornaTempArgs)
def retorna_temperatura(latitude:float, longitude:float):
    """Retorna a temperatura atual para um dada localidade"""

    URL = "https://api.open-meteo.com/v1/forecast"

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m",
        "forecast_days": 1
    }

    resposta = requests.get(URL, params=params)
    if resposta.status_code == 200:
        resultado = resposta.json()

        hora_atual = datetime.datetime.now(datetime.UTC).replace(tzinfo=None)
        lista_horas = [datetime.datetime.fromisoformat(temp_str) for temp_str in resultado["hourly"]["time"]]
        index_prox = min(range(len(lista_horas)), key=lambda i: abs(lista_horas[i] - hora_atual))

        temp_atual = resultado["hourly"]["temperature_2m"][index_prox]
        return temp_atual

    else:
        raise Exception(f"Erro ao obter a temperatura na API {URL}: {resposta.status_code}")

c:\Users\johns\OneDrive\Documentos\Projetos\GitHub\Regdoor_Project\venv\Lib\site-packages\IPython\core\interactiveshell.py:3579: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
retorna_temperatura.args_schema.schema()["properties"]

{'latitude': {'title': 'Latitude',
  'description': 'Latitude da localidade para a qual deseja obter a temperatura',
  'type': 'number'},
 'longitude': {'title': 'Longitude',
  'description': 'Longitude da localidade para a qual deseja obter a temperatura',
  'type': 'number'}}

In [4]:
retorna_temperatura.invoke({"latitude":-30, "longitude": -50})

28.6

## Criando tool do wikipedia

In [5]:
import wikipedia
wikipedia.set_lang("pt")

query = "Isaac Asimov"

@tool
def busca_wikipedia(query:str):
    """Faz busca no wikipedia e retorna resumos de paginas para a query"""
    titulos_paginas = wikipedia.search(query)
    resumos = []
    for titulo in titulos_paginas[:3]:
        try:
            wiki_page = wikipedia.page(title=titulo, auto_suggest=True)
            resumos.append(f"Titulo da pagina: {titulo} \n Resumo: {wiki_page.summary}")
        except:
            pass
    if not resumos:
        return "Nenhuma informação encontrada"
    else:
        return "\n\n".join(resumos)

In [6]:
busca_wikipedia("Newton")

C:\Users\johns\AppData\Local\Temp\ipykernel_1812\1677674536.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  busca_wikipedia("Newton")


'Titulo da pagina: Isaac Newton \n Resumo: Sir Isaac Newton PRS (Woolsthorpe-by-Colsterworth, 25 de dezembro de 1642jul./ 4 de janeiro de 1643greg. – Kensington, 20 de março de 1727jul./ 31 de março de 1727greg) foi um matemático, físico, astrônomo, teólogo e autor inglês (descrito em seus dias como um "filósofo natural") amplamente reconhecido como um dos cientistas mais influentes de todos os tempos e como uma figura-chave na Revolução Científica. Seu livro Philosophiæ Naturalis Principia Mathematica (Princípios Matemáticos da Filosofia Natural), publicado pela primeira vez em 1687, lançou as bases da mecânica clássica. Newton também fez contribuições seminais à óptica e compartilha crédito com Gottfried Wilhelm Leibniz pelo desenvolvimento do cálculo infinitesimal.\nEm Principia, Newton formulou as leis do movimento e da gravitação universal, que criaram o ponto de vista científico dominante até serem refinadas pela teoria da relatividade de Albert Einstein. Usou sua descrição matem

In [7]:
busca_wikipedia.invoke({"query": "Isaac Asimov"})

'Titulo da pagina: Isaac Asimov \n Resumo: Isaac Asimov (em russo: Исаак Юдович Озимов; romaniz.: Isaak Yudavich Azimov; Petrovichi, Rússia Soviética, atual Rússia, 2 de janeiro de 1920 — Brooklyn, 6 de abril de 1992) foi um escritor e bioquímico russo-americano, autor de obras de ficção científica e divulgação científica.\nAsimov é considerado um dos mestres da ficção científica e, junto com Robert A. Heinlein e Arthur C. Clarke, foi considerado um dos "três grandes" dessa área da literatura. A obra mais famosa de Asimov é a Série da Fundação, também conhecida como Trilogia da Fundação, que faz parte da série do Império Galáctico e que logo combinou com a Série Robôs. Também escreveu obras de mistério e fantasia, assim como uma grande quantidade de não-ficção. No total, escreveu ou editou mais de 500 volumes, aproximadamente 90 000 cartas ou postais, e tem obras em cada categoria importante do sistema de classificação bibliográfica de Dewey, exceto em filosofia.\nA maioria de seus liv

#### Testando a tool

In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function

prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente rude e grosso chamado Vagão"),
    ("user", "{input}")
])
chat = ChatOpenAI()

tools = [convert_to_openai_function(busca_wikipedia), convert_to_openai_function(retorna_temperatura)]
chain = prompt | chat.bind(functions=tools)
chain.invoke({"input": "Qual a temperatura de são paulo"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-23.5505,"longitude":-46.6333}', 'name': 'retorna_temperatura'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 141, 'total_tokens': 169, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-84cecbc8-b74b-4680-a8fc-8f4d15893a23-0', usage_metadata={'input_tokens': 141, 'output_tokens': 28, 'total_tokens': 169, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})